# RDS to S3

### Introduction

In the last set of lessons, we saw copy data from a CSV file in S3, and move it to our database in redshift.  But remember that our data does not start out in S3.  Instead, it will generally start from our RDS database, and from there we'll move the data over to a CSV file in S3 to ultimately move over to redshift. 

Let's take another look at the diagram illustrating our data flow.

<img src="./rds_ec2_s3.jpg" width="70%">

As we can see, to coordinate the flow of data from RDS to S3, we use an ETL server to perform commands moving data from RDS to S3, and ultimately to redshift.

In this lesson, we'll use our local laptop to copy our data first to our laptop and, from there, over to our S3 buckets.  Let's get started.

### Our Current Setup

Now, let's imagine that we already have our RDS instance setup.  We are using our foursquare-api database, which has the following tables.

> <img src="./display_tables.png" width="60%">

### Exporting Data to Our Local Computer

Imagine that we want to begin by exporting data from our postgres database to our local machine.  Let's choose the informatioon in the states table as data to move.

> <img src="./display-select-states.png" width="60%">

Now we can export information from this table onto our local computer as a CSV with the following.

```bash
psql -U postgres --host foursquare-flask-api.cbdkozm37vkd.us-east-1.rds.amazonaws.com -c "\copy (Select * From states) To 'states.csv' With CSV;"

```

So here we use the `\copy` command to `SELECT` information from the proper data table and then specify the file that we want to load it into.  The `WITH` clause is where we place any optional arguments like here the format of CSV.

> Other optional arguments can be in the [postgres documentation for copy](https://www.postgresql.org/docs/9.2/sql-copy.html).

### Moving to S3

Now if we take another look at our diagram, at this point, we have just exported data from our RDS instance over to our local machine with the postgres command of `\copy (Select * From states) To 'states.csv' With CSV;`

<img src="./rds_ec2_s3.jpg" width="60%">

The next step is to then export this CSV file over to our S3 file storage.  We can do export our data through a one line statement in the AWS command line, but first, we'll need to make sure we have the proper permissions to perform this upload.

To do so, navigate to the s3 dashboard.

> <img src="./s3-other-dash.png" width="60%">

And select the name of the bucket that you would like to upload the csv files to.  Here, it's `jigsaw-sample-data`.  Then we'll select permissions, and the first step we can do is to enable public access.

> We can eventually change this to only allow access from a specific IP address or EC2 instance, but for now this is fine.

> <img src="./sample-data-public.png" width="80%">

From there, the next step is to paste in the following as the bucket policy.

> The only information, we may need to alter is the resource -- change it to the name of your bucket, if different.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicRead",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3:PutObject",
                "s3:PutObjectAcl",
                "s3:GetObject",
                "s3:GetObjectAcl",
                "s3:DeleteObject"
            ],
            "Resource": "arn:aws:s3:::jigsaw-sample-data/*"
        }
    ]
}
```

> <img src="./bucket-policy.png" width="60%">

> FYI: `acl` stands for access control list.

Now that the permissions are properly setup, we can move to the command to take our data from our local machine and upload it into into our bucket.

<img src="./states-csv.png">

> `aws s3 cp states.csv s3://jigsaw-sample-data/states.csv --acl public-read`

And then we can see that we have the `states.csv` file located in the specified bucket.

### Summary

In this lesson, we saw the procedure for copying our data from an RDS instance, to a CSV file on an external computer, and then uploading the CSV file to S3.  

To copy the data from our RDS instance to the ETL server, we ran the following:

```bash
psql -U postgres --host foursquare-flask-api.cbdkozm37vkd.us-east-1.rds.amazonaws.com -c "\copy (Select * From states) To 'states.csv' With CSV;"
```

And this created a new `states.csv` file on our computer with data from our RDS instance.

Then came uploading this file to our `s3` bucket.  First, we set permissions on our S3 bucket to enable public access, and set a bucket policy that gave `put` access to add additional objects to the bucket.  After the permissions were set up, we were able to upload our csv file into the bucket with the following: 

`aws s3 cp states.csv s3://jigsaw-sample-data/states.csv --acl public-read`

### Resources




[S3 tutorial](http://boto.cloudhackers.com/en/latest/s3_tut.html)

[Good tutorial RDS to S3](https://www.sqlshack.com/integrating-aws-s3-buckets-with-aws-rds-sql-server/)